<a href="https://colab.research.google.com/github/coder-penguin/ML/blob/master/tutorial/keras_tutorial_gan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Sample code for image generation with GAN

Load libraries

In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import os
from PIL import Image, ImageOps
import subprocess
import glob
import matplotlib.image as mpimg
from tqdm import tqdm

from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score
from sklearn.externals import joblib

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Activation, Flatten, Conv2D, MaxPooling2D, Reshape, UpSampling2D, Input
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler, Callback
from tensorflow.keras.utils import plot_model, model_to_dot, to_categorical
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, load_img, img_to_array
from tensorflow.keras import backend as K
from IPython.display import SVG, clear_output

import shutil

from google.colab import drive
from google.colab import files

print(tf.__version__)

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


2.2.0-rc2


In [2]:
#check GPU
tf.test.gpu_device_name()

'/device:GPU:0'

Read data

In [3]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp '/content/drive/My Drive/img_extracted.zip' .
!unzip -q 'img_extracted.zip'
!rm 'img_extracted.zip'

In [5]:
!ls ./img_extracted

fake  real


In [6]:
!mv train img_extracted/real/real

mv: cannot stat 'train': No such file or directory


In [0]:
#move only real images
!mkdir train
!mv img_extracted/real/real train/

In [0]:
#get image names
input_directry = './train/real/'
images = glob.glob(input_directry+'*.*')

In [0]:
def show_images(directry, with_title=False):
  images = glob.glob(directry+'*.*')
  plt.figure(figsize=(28, 12))
  for i in range(12):
    if i < len(images):
      plt.subplot(3, 4, i+1)
      img = mpimg.imread(images[i])
      imgplot = plt.imshow(img)
      plt.axis('off')
      if with_title:
        plt.title(images[i])

In [0]:
#show input images
show_images(input_directry)

Set data generator

In [0]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        width_shift_range=0.1,
        height_shift_range=0.1,
        channel_shift_range=20.0,
        horizontal_flip=True,
        vertical_flip=False
        )

In [12]:
train_generator = train_datagen.flow_from_directory(
        './train/', #A subdirectory under the directory is required.
        target_size=(32, 32),
        batch_size=32,
        color_mode='grayscale',
        class_mode=None)

Found 270 images belonging to 1 classes.


In [13]:
train_generator.class_indices

{'real': 0}

###Train model

Three types of losses for GANs

There were proposed numerous loss functions to train GANs. In this notebook we have implemented three the most popular choices:

`KL`:


$$\mathcal{L}_g = \log(1 - \mathrm{discriminator}(\mathrm{gen}))$$

$$\mathcal{L}_d = - \log(\mathrm{discriminator}(\mathrm{real})) - \log(1 - \mathrm{discriminator}(\mathrm{gen}))$$


`REVERSED_KL`

$$\mathcal{L}_g = - \log(\mathrm{discriminator}(\mathrm{gen}))$$

$$\mathcal{L}_d = - \log(\mathrm{discriminator}(\mathrm{real})) - \log(1 - \mathrm{discriminator}(\mathrm{gen}))$$


`WASSERSTEIN`

$$\mathcal{L}_g = - \mathrm{discriminator}(\mathrm{gen})$$

$$\mathcal{L}_d = \mathrm{discriminator}(\mathrm{gen}) - \mathrm{discriminator}(\mathrm{real})$$

In [0]:
class GANLosses(object):
  def __init__(self, loss_type):
    self.loss_type = loss_type
   
  def d_loss(self, y_real, y_gen):
    eps = 1e-10
    if self.loss_type in ['KL', 'REVERSED_KL']: 
      loss = - K.mean(K.log(y_real + eps)) - K.mean(K.log(1 - y_gen + eps))
    elif self.loss_type == 'WASSERSTEIN':
      loss = - (K.mean(y_real) - K.mean(y_gen))
    return loss

  def g_loss(self, y_real, y_gen):
    eps = 1e-10
    if self.loss_type == 'KL': 
      loss = K.mean(K.log(1 - y_gen + eps))  
    elif self.loss_type == 'REVERSED_KL':
      loss = - K.mean(K.log(y_gen + eps))
    elif self.loss_type == 'WASSERSTEIN':
      loss = - K.mean(y_gen)
    return loss

  def calc_gradient_penalty(self, tape, discriminator, img_gen, img_real, lambda_reg = .1):
    #https://arxiv.org/abs/1704.00028
    with tape:
      alpha = np.random.uniform(size = (img_real.shape[0], 1, 1, 1))
      img_interpolates = (alpha * img_real + ((1 - alpha) * img_gen))
      y_interpolates = discriminator(img_interpolates)
    gradients = tape.gradient(y_interpolates, [img_interpolates])[0]
    gradients = K.sqrt(K.sum(K.square(gradients), axis=[1,2,3]))
    gradient_penalty = K.mean(K.square(gradients -1)) * lambda_reg
    return gradient_penalty
    
  def calc_zero_centered_gradient_penalty(self, tape, discriminator, img_gen, img_real, gamma_reg = .1):
    #https://arxiv.org/abs/1705.09367
    with tape:
      img_real += img_gen * 0
      y_real = discriminator(img_real)
    gradients = tape.gradient(y_real, [img_real])[0]
    gradients = K.sqrt(K.sum(K.square(gradients), axis=[1,2,3]))
    zc_gradient_penalty = gamma_reg / 2 * K.mean(K.square(gradients))
    
    return zc_gradient_penalty

In [0]:
class GAN(object):
  def __init__(self, discriminator, generator, optimizer_d, optimizer_g, noise_dim, k_d=10, k_g=1, 
                 loss_type='KL', penalty=None, instance_noise=False, lipschitz_weights=False, clip_lower=-0.1, clip_upper=0.1, verbose=0):
    self.discriminator = discriminator
    self.generator = generator
    self.noise_dim = noise_dim
    self.k_d = k_d
    self.k_g = k_g
    self.ganlosses = GANLosses(loss_type)
    self.penalty = penalty #GRAD, ZERO_CENTERED_GRAD 
    self.instance_noise = instance_noise 
    self.lipschitz_weights = lipschitz_weights
    self.clip_lower = clip_lower
    self.clip_upper = clip_upper

    #compile discriminator
    self.discriminator.compile(loss=self.ganlosses.d_loss, optimizer=optimizer_d) #is trainable
    print('discminator was sucessfully compiled!')

    #compile combined model
    self.discriminator.trainable = False #discriminator in the combined model must not be trainable
    self.combined = Sequential([self.generator, self.discriminator], name='combined')
    self.combined.compile(loss=self.ganlosses.g_loss, optimizer=optimizer_g)
    print('generator was sucessfully compiled!')

    self.discriminator.trainable = True #need to be True to get trainable_weights

    if verbose == 1 :
      self.discriminator.summary()
      self.combined.summary()

  def fit(self, dataset=None, epochs=1, steps_per_epoch=1, sample_interval=50):
    
    dis_loss_epochs= []
    gen_loss_epochs = []
    real_accuracy_epochs = [] 
    generated_accuracy_epochs = []

    for epoch in tqdm(range(epochs)):
      dis_loss = 0
      gen_loss = 0
      real_accuracy = Accuracy()
      generated_accuracy = Accuracy()
      
      for step, img_data in enumerate(dataset):
        if step == steps_per_epoch:
          break
                
        # Optimize D
        for _ in range(self.k_d):
          img_b = img_data * 2 - 1. #[0, 1] --> [-1, 1]
          noise = np.random.randn(len(img_b), self.noise_dim)

          if self.instance_noise :
            img_b = self.add_instance_noise(img_b)

          with tf.GradientTape() as tape1, tf.GradientTape() as tape2:
            y_b = self.discriminator(img_b)
            img_gen = self.combined.layers[0](noise)
            if self.instance_noise:
              img_gen = self.add_instance_noise(img_gen)
            y_gen = self.combined.layers[1](img_gen)
            loss = self.discriminator.loss(y_b, y_gen)
          
          #caluculate penalty
          if self.penalty == 'GRAD':
            grad_penalty = self.ganlosses.calc_gradient_penalty(tape2, self.discriminator, img_gen, img_b)  
            with tape1:   
              loss += grad_penalty           
          elif self.penalty == 'ZERO_CENTERED_GRAD':
            grad_penalty = self.ganlosses.calc_zero_centered_gradient_penalty(tape2, self.discriminator, img_gen, img_b)
            with tape1:   
              loss -= grad_penalty
          
          #update weights manually
          gradients = tape1.gradient(loss, self.discriminator.trainable_weights)
          self.discriminator.optimizer.apply_gradients(zip(gradients, self.discriminator.trainable_weights))

          if self.lipschitz_weights :                    
            for layer in self.discriminator.layers:
              weights = layer.get_weights()
              weights = [np.clip(w, self.clip_lower, self.clip_upper) for w in weights]
              layer.set_weights(weights)

        dis_loss += loss

        # Optimize G
        for _ in range(self.k_g):
          img_b = img_data * 2 - 1. #[0, 1] --> [-1, 1]
          noise = np.random.randn(len(img_b), self.noise_dim)

          if self.instance_noise:
            img_b = self.add_instance_noise(img_b)

          #get loss
          with tf.GradientTape() as tape:
            y_b = self.discriminator(img_b)
            if self.instance_noise:
              img_gen = self.combined.layers[0](noise)
              img_gen = self.add_instance_noise(img_gen)
              y_gen = self.combined.layers[1](img_gen)
            else:
              y_gen = self.combined(noise)
            loss = self.combined.loss(y_b, y_gen)
  
          #update weights manually
          gradients = tape.gradient(loss, self.combined.trainable_weights)
          self.combined.optimizer.apply_gradients(zip(gradients, self.combined.trainable_weights))

        gen_loss += loss

        # update accuracy
        #accuracy for real images
        y_pred = [1 if y_b[i]>0.5 else 0 for i in range(len(y_b))]
        y_true = [1 for _ in range(len(y_b))] 
        real_accuracy.update_state(y_true, y_pred)

        #accuracy for generated images
        y_pred = [1 if y_gen[i]>0.5 else 0 for i in range(len(y_gen))]
        y_true = [0 for _ in range(len(y_gen))]       
        generated_accuracy.update_state(y_true, y_pred)     

      dis_loss /= steps_per_epoch
      gen_loss /= steps_per_epoch

      dis_loss_epochs.append(dis_loss)
      gen_loss_epochs.append(gen_loss)
      real_accuracy_epochs.append(real_accuracy.result().numpy())
      generated_accuracy_epochs.append(generated_accuracy.result().numpy())

      clear_output()
      plt.figure(figsize=(12, 12))
      plt.plot(dis_loss_epochs, label='dis_loss')
      plt.plot(gen_loss_epochs, label='gen_loss')
      plt.legend()
      plt.show()
        
      plt.figure(figsize=(12, 12))
      plt.plot(real_accuracy_epochs, label='real_accuracy')
      plt.plot(generated_accuracy_epochs, label='generated_accuracy')
      plt.plot()
      plt.legend()
      plt.show()

      #print(dis_loss, gen_loss)

      if epoch % sample_interval == 0:
        self.sample_images(epoch)
  
  def add_instance_noise(self, data, std=0.01):
    # https://arxiv.org/abs/1610.04490
    return data + np.random.normal(0, std, size=data.shape)

  def sample_images(self, epoch):
    row, column = 3, 4
    noise = np.random.randn(row * column, self.noise_dim)
    gen_imgs = self.combined.layers[0](noise)   
    gen_imgs = 0.5 * gen_imgs + 0.5 # Rescale images [-1, 1] --> [0, 1]

    plt.figure(figsize=(28, 12))
    fig, axs = plt.subplots(row, column)
    count = 0
    for i in range(row):
      for j in range(column):
        axs[i,j].imshow(gen_imgs[count, :,:,0], cmap='gray')
        axs[i,j].axis('off')
        count += 1

    if not os.path.exists('images'): 
      subprocess.call('mkdir images', shell=True)
    fig.savefig('images/images_epoch%d.png' % epoch)
    plt.close()

In [0]:
def build_discriminator():
  activation = 'elu'
  model = Sequential([
    Conv2D(32,3, input_shape=(32, 32, 1)),#filters, kernel_size
    Activation(activation),
    #MaxPooling2D(pool_size=4),
    Conv2D(64,3),
    Activation(activation),
    MaxPooling2D(pool_size=4),
    Dropout(0.2),
    #Conv2D(64,3),
    #Activation(activation),
    #MaxPooling2D(pool_size=4),
    #Dropout(0.2),

    Flatten(),
    Dense(128),
    BatchNormalization(),
    Activation(activation),
    Dropout(0.5),
    Dense(32, activation=activation),
    Dense(1, activation='sigmoid')
  ], name='discriminator')

  #do not compile this here

  return model

In [0]:
def build_generator(noise_dim):
  activation = 'elu'
  model = Sequential([
    Dense(8 * 8 * 128, input_dim=noise_dim),
    Activation(activation),
    Reshape((8, 8, 128)),
    UpSampling2D(),
    Conv2D(128, kernel_size=4, padding='same'),
    BatchNormalization(momentum=0.8),
    Activation(activation),
    UpSampling2D(),
    Conv2D(64, kernel_size=4, padding='same'),
    BatchNormalization(momentum=0.8),
    Activation(activation),
    Conv2D(1, kernel_size=4, padding='same'),#gray-scale
    Activation('tanh')
  ], name='generator')

  #do not compile this here

  return model

In [26]:
noise_dim = 100
optimizer_d = SGD(lr=0.005, nesterov=True)
optimizer_g = SGD(lr=0.005, nesterov=True)
discriminator = build_discriminator()
generator = build_generator(noise_dim)
gan = GAN(discriminator, generator, optimizer_d, optimizer_g, noise_dim, k_d=5, instance_noise=True)
#gan = GAN(discriminator, generator, optimizer_d, optimizer_g, noise_dim, k_d=3, instance_noise=True, loss_type='WASSERSTEIN', penalty='GRAD')

discminator was sucessfully compiled!
generator was sucessfully compiled!


In [0]:
gan.fit(train_generator, epochs=1000, steps_per_epoch=100)

###Show generated images

In [48]:
sample_epoch = 10
gan.sample_images(sample_epoch)

<Figure size 2016x864 with 0 Axes>

In [0]:
images = glob.glob('./images/*.*')
plt.figure(figsize=(12, 8))
img = mpimg.imread('./images/images_epoch%d.png'%sample_epoch)
imgplot = plt.imshow(img)
plt.axis('off')

###Save

In [0]:
# save the model
disc_filename = '/content/drive/My Drive/discriminator.h5'
gan.discriminator.save(disc_filename)
gen_filename = '/content/drive/My Drive/generator.h5'
gan.generator.save(gen_filename)

In [23]:
!zip -r /content/gan_images.zip ./images

  adding: images/ (stored 0%)
  adding: images/images_epoch10.png (deflated 6%)
  adding: images/images_epoch0.png (deflated 5%)


In [0]:
files.download('./gan_images.zip')